In [16]:
import warnings
import os
import sys


warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [17]:
import pandas as pd
from bertopic import BERTopic
from nltk.corpus import stopwords
import nltk
import os
from src.main.utilities import utils

os.environ["TOKENIZERS_PARALLELISM"] = "false" 

#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

#read and clean dataset
df = pd.read_json('../../dataset/News_Category_Dataset.json', lines=True)
df = utils.clean(df)

print(len(df))

# Remove stopwords from dataset
df['short_description'] = df['short_description'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))

# Extract 100000 samples of short_description and category for Bertopic
docs = df['short_description'].tolist()[:50000]
cat = df['category'].tolist()[:50000]

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, nr_topics=21)

# Fit BERTopic model and extract topics
topics, _ = topic_model.fit_transform(docs)



In [18]:
#plot founded topics
print(topic_model.get_topic_info())

In [19]:

# Recreate dataframe for counting unique categories per topic
df = pd.DataFrame({'Document': docs, 'Category': cat, 'Topic': topics})

df = df.groupby('Topic')['Category'].nunique()



In [20]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))  
plt.bar(df.index, df.values, color='skyblue')

#plt.xticks(df.index)

plt.xlabel('Topics')
plt.ylabel('Number of Unique Categories')
plt.title('# Unique Categories per Topic')

plt.show()